## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Cedric Sadeu (1869737): 1/3

Mamoudou Sacko (1924187): 1/3

Oumayma Messoussi (2016797): 1/3



## Support de google Colab



In [1]:
#!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

The history saving thread hit an unexpected error (DatabaseError(&#39;database disk image is malformed&#39;)).History will not be written to the database.


In [2]:
#! tar -xzf aclImdb_v1.tar.gz
#! tar -xzf MEN.tar.gz
#! mkdir -p vsm

## Librairies externes

In [3]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')

In [4]:
from collections import Counter, defaultdict
from itertools import chain
from bs4 import BeautifulSoup
import csv
import itertools
import numpy as np
import os
import pandas as pd
import glob
import copy
import re
from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [5]:
DIRNAME_ACL =  "aclImdb/"
#DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN/")
#DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [6]:
train_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}
test_data = {'id': [], 'text': [], 'rating': [], 'sentiment': []}

for name in ['train/pos', 'train/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        train_data['id'].append(review_id)
        train_data['text'].append(content)
        train_data['rating'].append(rating)
        train_data['sentiment'].append(sentiment)
                
for name in ['test/pos', 'test/neg']:
    sentiment = name.split('/')[1]
    for file in glob.glob(DIRNAME_ACL+name+"/*.txt"):
        review_id = int(file.split("/")[-1].split(".")[0].split("_")[0])
        rating = int(file.split("/")[-1].split(".")[0].split("_")[1]) / 10
        f = open(file, encoding="utf8")
        content = f.read()
        
        test_data['id'].append(review_id)
        test_data['text'].append(content)
        test_data['rating'].append(rating)
        test_data['sentiment'].append(sentiment)

train_df = pd.DataFrame (train_data, columns = ['id', 'text', 'rating', 'sentiment'])
test_df = pd.DataFrame (test_data, columns = ['id', 'text', 'rating', 'sentiment'])

In [7]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,0,Bromwell High is a cartoon comedy. It ran at t...,0.9,pos
1,10000,Homelessness (or Houselessness as George Carli...,0.8,pos
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,1.0,pos
3,10002,This is easily the most underrated film inn th...,0.7,pos
4,10003,This is not the typical Mel Brooks film. It wa...,0.8,pos


In [8]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,0,I went and saw this movie last night after bei...,1.0,pos
1,10000,Actor turned director Bill Paxton follows up h...,0.7,pos
2,10001,As a recreational golfer with some knowledge o...,0.9,pos
3,10002,"I saw this film in a sneak preview, and it is ...",0.8,pos
4,10003,Bill Paxton has taken the true story of the 19...,0.8,pos


In [9]:
def preprocess(review):
    soup = BeautifulSoup(review, "html.parser")
    text = soup.get_text()
    stop_words = set(all_stopwords.words('english'))
    tokens = nltk.word_tokenize(text.lower())
    return [w for w in tokens if w.isalpha() and not w in stop_words] 

In [10]:
train_df['text'] = train_df['text'].apply(preprocess)
test_df['text'] = test_df['text'].apply(preprocess)

In [11]:
print(train_df.shape)
train_df.head()

(25000, 4)


,id,text,rating,sentiment
0,0,"[bromwell, high, cartoon, comedy, ran, time, p...",0.9,pos
1,10000,"[homelessness, houselessness, george, carlin, ...",0.8,pos
2,10001,"[brilliant, lesley, ann, warren, best, dramati...",1.0,pos
3,10002,"[easily, underrated, film, inn, brooks, cannon...",0.7,pos
4,10003,"[typical, mel, brooks, film, much, less, slaps...",0.8,pos


In [12]:
print(test_df.shape)
test_df.head()

(25000, 4)


,id,text,rating,sentiment
0,0,"[went, saw, movie, last, night, coaxed, friend...",1.0,pos
1,10000,"[actor, turned, director, bill, paxton, follow...",0.7,pos
2,10001,"[recreational, golfer, knowledge, sport, histo...",0.9,pos
3,10002,"[saw, film, sneak, preview, delightful, cinema...",0.8,pos
4,10003,"[bill, paxton, taken, true, story, us, golf, o...",0.8,pos


In [13]:
print("""TRAIN\n\t - le nombre total de phrases d’entrainement: %d
              \n\t - le nombre total de phrases d’entrainement positives: %d
              \n\t - le nombre total de phrases d’entrainement négatives: %d\n""" % 
      (train_df.shape[0], train_df[train_df["sentiment"] == "pos"].shape[0], 
       train_df[train_df["sentiment"] == "neg"].shape[0]))

print("""TEST\n\t - le nombre total de phrases de test: %d
              \n\t - le nombre total de phrases de test positives: %d
              \n\t - le nombre total de phrases de test négatives: %d\n""" % 
      (test_df.shape[0], test_df[test_df["sentiment"] == "pos"].shape[0], 
       test_df[test_df["sentiment"] == "neg"].shape[0]))

TRAIN
	 - le nombre total de phrases d’entrainement: 25000
              
	 - le nombre total de phrases d’entrainement positives: 12500
              
	 - le nombre total de phrases d’entrainement négatives: 12500

TEST
	 - le nombre total de phrases de test: 25000
              
	 - le nombre total de phrases de test positives: 12500
              
	 - le nombre total de phrases de test négatives: 12500



**b)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [14]:
def build_voc(corpus, unk_cutoff=5):
    unigrams = defaultdict(int)
    for review in corpus:
        for word in review:
            unigrams[word] += 1

    print("Unigrammes: " + str(len(unigrams)))

    unigrams = {k: v for k, v in sorted(unigrams.items(), key=lambda item: item[1], reverse=True) if v >= 5}

    print("Taille du vocab (apres filtrage freq > 5): " + str(len(unigrams)))

    f = open('vocab.txt', 'w')
    for w in unigrams.keys():
        f.write(w + '\n')

    return unigrams

In [15]:
train_unigrams = build_voc(train_df['text'])

Unigrammes: 72125
Taille du vocab (apres filtrage freq &gt; 5): 27791


## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [16]:
def get_neighbours(list_words, index, N, is_distance):
    result = []
    try:
        word = list_words[index]
    except:
        return result
    max_index =  index + N
    if max_index >= len(list_words):
        max_index = len(list_words) - 1
    min_index = index - N
    if min_index < 0:
        min_index = 0
    if is_distance:
        while(min_index <= max_index):
            if list_words[min_index] != word:
                result.append(((word, list_words[min_index]), 1 / abs(min_index - index)))
            min_index += 1
    else:
        while(min_index <= max_index):
            if list_words[min_index] != word:
                result.append((word, list_words[min_index]))
            min_index += 1
    return result

def neighbours_grapper(list_words, word, N, is_distance):
    #if list_words is empty return
    if not list_words:
        return []
    #find all indexes of word
    indexes = [i for i, w in enumerate(list_words) if w == word]
    #if indexes is empty return
    if not indexes:
        return []
    result = []
    for i in indexes:
        result.extend(get_neighbours(list_words, i, N, is_distance))
    return result

def neighbours_corpus(corpus, N, unigramme_list, is_distance):
    result = []
    for word in unigramme_list:
        for sentence in corpus:
            result.extend(neighbours_grapper(re.split(r'\s', sentence), word, N, is_distance))
    return result

def dict_tuple_creator(tuple_list):
    dict_tuple = Counter(tuple_list)
    return dict(dict_tuple)

def matrix_creator(dict_tuple, unigramme_list):
    # avoir un dict avec key (mot1 , mot2) et frequence comme value
    # creer une matrice 5000 * 5000 et la remplir de zero
    # parcourir le dictionnaire:
    #   pour chaque key (mot1 , mot2): find la position du mot1 et du mot2 dans la matrice
    #                           pour avoir ces coordonnes (pos1, pos2) dans la matrice
    # mettre la value correspondante a cette key a (pos1, pos2) dans la matrice
    N = len(unigramme_list)
    mat_result = []
    count = 0
    while count < N :
        mat_result.append([0] * N)
        count+=1
    
    for key in dict_tuple.keys():
        #if key[0] == key[1]:
        try:
            x = unigramme_list.index(key[0])
            y = unigramme_list.index(key[1])
        except ValueError:
            continue
        mat_result[x][y] = dict_tuple[key]
    
    return mat_result

def mat_list_to_df(mat_result, unigramme_list):
    arr = np.array(mat_result)
    t_arr = arr.T
    arr_list = t_arr.tolist()
    dict_result = {}
    for i, word in enumerate(unigramme_list):
        dict_result[word] = arr_list[i]
    df = pd.DataFrame(data = dict_result, index = unigramme_list)
    return df

In [17]:
train_top_5000 = list(train_unigrams.keys())[:5000] # or read first 5000 lines from vocab.txt
train_5000_df = pd.DataFrame(train_top_5000, columns=['unigram'])
train_5000_df.shape
train_5000_df.head()

corpus = list(train_df['text'])
corpus = [' '.join(sentence) for sentence in corpus]
print(len(corpus))



25000


In [18]:
%%time
tuple_list = neighbours_corpus(corpus, 5, train_top_5000, False)

CPU times: user 42min 5s, sys: 1.77 s, total: 42min 7s
Wall time: 42min 7s


In [19]:
dict_tuple = dict_tuple_creator(tuple_list)
mat_result = matrix_creator(dict_tuple, train_top_5000)
df = mat_list_to_df(mat_result, train_top_5000)
df.to_csv('tp2_mat5.csv')

**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

In [20]:
def dict_tuple_creator_d(tuple_list):
    res = list(zip(*tuple_list))
    tup = list(res[0])
    dict_tup = Counter(tup)
    for key in dict_tup.keys():
        temp = [item[1] for item in tuple_list if item[0] == key]
        dict_tup[key] = sum(temp)
    return dict(dict_tup)





In [21]:
tuple_list_d = neighbours_corpus(corpus, 5, train_top_5000, True)
dict_tuple_d = dict_tuple_creator_d(tuple_list_d)
mat_result_d = matrix_creator(dict_tuple_d, train_top_5000)
df_d = mat_list_to_df(mat_result_d, train_top_5000)
df_d.to_csv('tp2_mat5_scaled.csv')

KeyboardInterrupt: 

**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [ ]:
def probability_matrix(mat_Result):
    total = 0
    for line in mat_Result:
        total += sum(line)
    arr = np.array(mat_Result)
    arr_prob = np.divide(arr, total)
    return arr_prob.tolist()

def ppmi_pmi(df_Result, is_ppmi):
    temp = df_Result.to_numpy()
    mat_Result = list(temp.tolist())
    mat_prob = probability_matrix(mat_Result)
    arr = np.array(mat_prob)
    sum_arr_column = arr.sum(axis=0)
    sum_arr_line = arr.sum(axis=1)
    for i, line in enumerate(mat_prob):
        for j, item in enumerate(line):
            if item == 0:
                mat_prob[i][j] = None
            elif is_ppmi:         
                mat_prob[i][j] = max(math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2), 0)
            else:
                mat_prob[i][j] = math.log(item / (sum_arr_column[j]*sum_arr_line[i]), 2)
    
    return mat_list_to_df(mat_prob, list(df_Result.columns))


#def pmi(df, flag=True):

**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

In [ ]:
result_ppmi_df = ppmi_pmi(df, True)
result_ppmi_df.to_csv('tp2_mat5_ppmi.csv')
result_pmi_df = ppmi_pmi(df, False)
result_pmi_df.to_csv('tp2_mat5_pmi.csv')

result_ppmi_sc_df = ppmi_pmi(df_d, True)
result_ppmi_sc_df.to_csv('tp2_mat5_scaled_ppmi.csv')
result_pmi_sc_df = ppmi_pmi(df_d, False)
result_pmi_sc_df.to_csv('tp2_mat5_scaled_pmi.csv')

## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [ ]:
def voisins(word, df, distfunc=cosine):

**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
def lsa(df, k=100):

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

f) Commentez vos résultats

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

**b)**	Commentez vos résultats d'évaluation